__title__      = ML in Action Chapter5 Code
__author__     = wgj
__createDate__ = 2018-10-16 19:39:25

In [3]:
from numpy import *
import random

def sigmoid(inX):
    return 1.0/(1+exp(-inX))

""" 梯度上升法 """
def gradAscent(dataMatIn, classLabels):
    """
    输入：1、dataMatIn——输入数据
         2、classLabels——类别标签
    输出：weights——回归系数
    """
    dataMatrix = mat(dataMatIn)             # 转换为NumPy矩阵类型
    labelMat = mat(classLabels).transpose() # 转换为NumPy矩阵类型并转置为列向量
    m,n = shape(dataMatrix)                 # 输入数据的行、列数
    alpha = 0.001                           # 初始化步进因子
    maxCycles = 500                         # 初始化迭代次数
    weights = ones((n,1))                   # 初始化回归系数，全1矩阵，n行1列
    for k in range(maxCycles):
        h = sigmoid(dataMatrix*weights)     # 计算对数几率，m行1列（括号内为矩阵乘）
        error = (labelMat - h)              # 类别差，m行1列
        weights = weights + alpha * dataMatrix.transpose()* error # 更新回归系数
    return weights

“梯度上升法”的不足在于每次更新回归系数时都要遍历整个数据集，计算复杂度高
因此，产生了“随机梯度上升法”，即每次仅使用1个样本点数据来更新回归系数

In [5]:
""" 随机梯度上升法 """
def stocGradAscent0(dataMatrix, classLabels):
    """
    输入：1、dataMatIn——输入数据
         2、classLabels——类别标签
    输出：weights——回归系数
    """
    m,n = shape(dataMatrix)
    alpha = 0.01
    weights = ones(n)   # 1行n列
    for i in range(m):  # 迭代次数等同于样本集数量
        h = sigmoid(sum(dataMatrix[i]*weights))  # 输入数据为array，此处为元素乘
        error = classLabels[i] - h
        weights = weights + alpha * error * dataMatrix[i]  # dataMatrix[i]为第i行数据，共n个数
    return weights

“随机梯度上升法”的不足在于回归系数存在周期震荡波动
因此，产生了“改进的随机梯度上升法”，改进之处有两点：
1、动态调整步进因子；
2、随机选取样本来更新回归系数。

In [11]:
""" 改进的随机梯度上升法 """
def stocGradAscent1(dataMatrix, classLabels, numIter=150):
    """
    输入：1、dataMatIn——输入数据
         2、classLabels——类别标签
         3、numIter——迭代次数（可选值，默认为150）
    输出：weights——回归系数
    """
    m,n = shape(dataMatrix)
    weights = ones(n)
    for j in range(numIter):
        for i in range(m):
            alpha = 4/(1.0+j+i)+0.0001    # alpha随着迭代递减（有常数存在，不会为0）
            randIndex = int(random.uniform(0,m))
            h = sigmoid(sum(dataMatrix[randIndex]*weights))
            error = classLabels[randIndex] - h
            weights = weights + alpha * error * dataMatrix[randIndex]
    return weights

In [7]:
""" 回归分类 """
def classifyVector(inX, weights):
    """
    输入：1、inX——待预测数据
         2、weights——回归系数
    输出：预测类别（0、1）
    """
    prob = sigmoid(sum(inX*weights))
    if prob > 0.5: return 1.0
    else: return 0.0

""" 从疝气病马预测死亡率 """
def colicTest():
    """
    输入：无
    输出：errorRate——错误率
    """
    frTrain = open('horseColicTraining.txt');  # 打开训练文件
    frTest = open('horseColicTest.txt')        # 打开测试文件
    trainingSet = []; trainingLabels = []      # 初始化训练集与标签
    for line in frTrain.readlines():
        currLine = line.strip().split('\t')    # 移除每行首尾空格并用tab分割数据
        lineArr =[]           # 保存一行训练数据
        for i in range(21):   # 每行22个数，前21个为特征
            lineArr.append(float(currLine[i]))  # 将字符串转换成浮点型
        trainingSet.append(lineArr)             # 向训练集中添加1行
        trainingLabels.append(float(currLine[21]))  # 向标签添加数据
    # 使用改进的梯度上升法计算回归系数
    trainWeights = stocGradAscent1(array(trainingSet), trainingLabels, 1000)
    errorCount = 0; numTestVec = 0.0    # 初始化错误数和测试样本数
    for line in frTest.readlines():
        numTestVec += 1.0
        currLine = line.strip().split('\t')
        lineArr =[]
        for i in range(21):
            lineArr.append(float(currLine[i]))
        if int(classifyVector(array(lineArr), trainWeights))!= int(currLine[21]):
            errorCount += 1
    errorRate = (float(errorCount)/numTestVec)
    print("本次测试的错误率为: %f" % errorRate)
    return errorRate

""" 多次测试 """
def multiTest():
    numTests = 10; errorSum = 0.0
    for k in range(numTests):
        errorSum += colicTest()
    print("%d 次迭代后，平均错误率为: %f" % (numTests,errorSum/float(numTests)))

In [12]:
multiTest()

本次测试的错误率为: 0.298507
本次测试的错误率为: 0.313433
本次测试的错误率为: 0.328358
本次测试的错误率为: 0.283582
本次测试的错误率为: 0.313433
本次测试的错误率为: 0.313433
本次测试的错误率为: 0.298507
本次测试的错误率为: 0.402985
本次测试的错误率为: 0.298507
本次测试的错误率为: 0.328358
10 次迭代后，平均错误率为: 0.317910
